In [1]:
# import sys
# import os

In [2]:
# sys.path.append('/home/super/waterffle/playlist')

In [3]:
# print(sys.path)

In [4]:
# from preprocessing import load_poly_encoder_dataset
# now_mfcc_list, next_mfcc_list, label_list = load_poly_encoder_dataset(2800)

In [5]:
# bsz * 10 개만 할당
# now_mfcc_list = torch.tensor(now_mfcc_list[:320])
# next_mfcc_list = torch.tensor(next_mfcc_list[:320])
# label_list = torch.tensor(label_list[:320])

In [6]:
# gpu 메모리 일부로 제한
# torch.cuda.set_per_process_memory_fraction(0.5,1)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
from preprocessing import load_poly_encoder_dataset

In [9]:
# device_num = 1

In [10]:
last_pad_length = 2800

In [11]:
cs, ns, labels = load_poly_encoder_dataset(last_pad_length)

====== Get MFCC with padding and crop======


100%|██████████████████████████████████████████████████████████| 32000/32000 [00:23<00:00, 1383.91it/s]


In [12]:
# bsz * 10 개만 할당
part = 320
cs = torch.tensor(cs[:part])
ns = torch.tensor(ns[:part])
labels = torch.tensor(labels[:part])

# cs = torch.tensor(cs)
# ns = torch.tensor(ns)
# labels = torch.tensor(labels)

In [13]:
cs = cs.view(-1, 1, last_pad_length, 40)
ns = ns.view(-1, 1, last_pad_length, 40)

In [14]:
cs.size(), ns.size()

(torch.Size([320, 1, 2800, 40]), torch.Size([320, 1, 2800, 40]))

In [15]:
cs[0].size()

torch.Size([1, 2800, 40])

-----------------------------

In [16]:
import logging

logger = logging.getLogger(__name__)

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Any, Dict, Optional, Tuple, Union
from __future__ import absolute_import, division, print_function, unicode_literals

from collections.abc import Iterable
from itertools import repeat

"""Near infinity, useful as a large penalty for scoring when inf is bad."""
NEAR_INF = 1e20
NEAR_INF_FP16 = 65504

def _pair(v):
    if isinstance(v, Iterable):
        assert len(v) == 2, "len(v) != 2"
        return v
    return tuple(repeat(v, 2))


def infer_conv_output_dim(conv_op, input_dim, sample_inchannel):
    sample_seq_len = 200
    sample_bsz = 10
    x = torch.randn(sample_bsz, sample_inchannel, sample_seq_len, input_dim)
    # N x C x H x W
    # N: sample_bsz, C: sample_inchannel, H: sample_seq_len, W: input_dim
    x = conv_op(x)
    # N x C x H x W
    x = x.transpose(1, 2)
    # N x H x C x W
    bsz, seq = x.size()[:2]
    per_channel_dim = x.size()[3]
    # bsz: N, seq: H, CxW the rest
    return x.contiguous().view(bsz, seq, -1).size(-1), per_channel_dim


'''
Two 2-D convolutional blocks, each with two conv. layers with kernel size=3, max-pooling kernel=2. The first block has 64 feature maps while the second has 128
'''
class ConvEncoder(nn.Module):
    """
    VGG motibated cnn module https://arxiv.org/pdf/1409.1556.pdf
    code : https://github.com/pytorch/fairseq/blob/c36294ea4fd35eac757f417de9668b32c57d4b3d/fairseq/modules/vggblock.py#L38
    Args:
        in_channels: (int) number of input channels (typically 1)
        out_channels: (int) number of output channels
        conv_kernel_size: convolution channels
        pooling_kernel_size: the size of the pooling window to take a max over
        num_conv_layers: (int) number of convolution layers
        input_dim: (int) input dimension
        conv_stride: the stride of the convolving kernel.
            Can be a single number or a tuple (sH, sW)  Default: 1
        padding: implicit paddings on both sides of the input.
            Can be a single number or a tuple (padH, padW). Default: None
        layer_norm: (bool) if layer norm is going to be applied. Default: False
    Shape:
        Input: BxCxTxfeat, i.e. (batch_size, input_size, timesteps, features)
        Output: BxCxTxfeat, i.e. (batch_size, input_size, timesteps, features)
    """
    def __init__(self, 
                 in_channels, 
                 out_channels,
                 conv_kernel_size,
                 num_conv_layers,
                 pooling_kernel_size,
                 input_dim=None, 
                 conv_stride=1,
                 padding=None,
                 layer_norm=True
                 ):
        assert (
            input_dim is not None
        ), "Need input_dim for LayerNorm and infer_conv_output_dim"
        super(ConvEncoder, self).__init__()
        
        conv_kernel_size = _pair(conv_kernel_size)
        pooling_kernel_size = _pair(pooling_kernel_size)
        padding = (
            tuple(e // 2 for e in conv_kernel_size)
            if padding is None
            else _pair(padding)
        )
        conv_stride = _pair(conv_stride)
        
        self.layers = nn.ModuleList()

        # input_channels=1 : mfcc는 2d이므로 채널을 1로 봐야함.
        # kernel size 3
        for layer in range(num_conv_layers):
            conv_op = nn.Conv2d(
                in_channels if layer == 0 else out_channels,
                out_channels,
                conv_kernel_size,
                stride=conv_stride,
                padding=padding,
            )
            self.layers.append(conv_op)
            if layer_norm:
                self.conv_output_dim, per_channel_dim = infer_conv_output_dim(
                    conv_op, input_dim, in_channels if layer == 0 else out_channels
                )
                self.layers.append(nn.LayerNorm(per_channel_dim))
                input_dim = per_channel_dim
            self.layers.append(nn.ReLU())
        
        if pooling_kernel_size is not None:
            # ceil_mode : when True, will use ceil instead of floor to compute the output shape
            pool_op = nn.MaxPool2d(kernel_size=pooling_kernel_size, ceil_mode=True)
            self.layers.append(pool_op)
            self.total_output_dim, self.output_dim = infer_conv_output_dim(
                pool_op, input_dim, out_channels
            )
                
    def forward(self, x):
        for i, _ in enumerate(self.layers):
            x = self.layers[i](x)
        return x


class BasicAttention(nn.Module):
    """
    Implements simple/classical attention.
    """

    def __init__(
        self,
        dim: int = 1,
        attn: str = 'cosine',
        residual: bool = False,
        get_weights: bool = True,
    ):
        super().__init__()
        if attn == 'cosine':
            self.cosine = nn.CosineSimilarity(dim=dim)
        self.attn = attn
        self.dim = dim
        self.get_weights = get_weights
        self.residual = residual

    def forward(
        self,
        xs: torch.Tensor,
        ys: torch.Tensor,
        mask_ys: Optional[torch.Tensor] = None,
        values: Optional[torch.Tensor] = None,
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """
        Compute attention.
        Attend over ys with query xs to obtain weights, then apply weights to
        values (ys if yalues is None)
        Args:
            xs: B x query_len x dim (queries)
            ys: B x key_len x dim (keys)
            mask_ys: B x key_len (mask)
            values: B x value_len x dim (values); if None, default to ys
        """
        bsz = xs.size(0)
        y_len = ys.size(1)
        x_len = xs.size(1)
        if self.attn == 'cosine':
            l1 = self.cosine(xs, ys).unsqueeze(self.dim - 1)
        else:
            l1 = torch.bmm(xs, ys.transpose(1, 2))
            if self.attn == 'sqrt':
                d_k = ys.size(-1)
                l1 = l1 / math.sqrt(d_k)
        if mask_ys is not None:
            attn_mask = (mask_ys == 0).view(bsz, 1, y_len)
            attn_mask = attn_mask.repeat(1, x_len, 1)
            l1.masked_fill_(attn_mask, neginf(l1.dtype))
        l2 = F.softmax(l1, dim=self.dim, dtype=torch.float).type_as(l1)
        if values is None:
            values = ys
        lhs_emb = torch.bmm(l2, values)

        # # add back the query
        if self.residual:
            lhs_emb = lhs_emb.add(xs)
        
        res = lhs_emb.squeeze(self.dim - 1)
        if self.get_weights:
            return res, l2
        else:
            return res
        

class PolyBasicAttention(BasicAttention):
    """
    Override basic attention to account for edge case for polyencoder.
    """

    def __init__(self, poly_type, n_codes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.poly_type = poly_type
        self.n_codes = n_codes

    def forward(self, *args, **kwargs):
        """
        Forward pass.
        Account for accidental dimensionality reduction when num_codes is 1 and the
        polyencoder type is 'codes'
        """
        lhs_emb = super().forward(*args, **kwargs)
        if self.poly_type == 'codes' and self.n_codes == 1 and len(lhs_emb.shape) == 2:
            lhs_emb = lhs_emb.unsqueeze(self.dim - 1)
        return lhs_emb
    

def neginf(dtype: torch.dtype) -> float:
    """
    Return a representable finite number near -inf for a dtype.
    """
    if dtype is torch.float16:
        return -NEAR_INF_FP16
    else:
        return -NEAR_INF
    

class AudioEncoder(nn.Module):
    '''
    code : https://github.com/pytorch/fairseq/blob/c36294ea4fd35eac757f417de9668b32c57d4b3d/examples/speech_recognition/models/vggtransformer.py#L271
    '''
    def __init__(self,
                 poly_n_codes, # m, the number of global contxt features
                 poly_attention_num_heads,
                 num_att_layers,
                 codes_attention_num_heads,
                 embed_dim, 
                
                 input_feat_per_channel,
                 num_conv_block,
                 num_conv_layers, 
                 in_channels, 
                 out_channels,
                 conv_kernel_size=3, 
                 pooling_kernel_size=2,
                 layer_norm=False,
                 dropout=0.1,
                 reduction_type='first' # first, avg, max
                ):
        super(AudioEncoder, self).__init__()
                
        self.in_channels = in_channels
        self.input_dim = input_feat_per_channel
        self.reduction_type = reduction_type
        
        self.conv_encoder_block = nn.ModuleList()
        
        for i in range(num_conv_block):
            self.conv_encoder_block.append(
                ConvEncoder(
                    in_channels=in_channels, 
                    out_channels=out_channels,
                    conv_kernel_size=conv_kernel_size,
                    num_conv_layers=num_conv_layers,
                    pooling_kernel_size=pooling_kernel_size,
                    input_dim=input_feat_per_channel,
                    layer_norm=layer_norm
                    )
                )
            in_channels = out_channels
            input_feat_per_channel = self.conv_encoder_block[-1].output_dim
        
        self.conv_encoder_block = nn.Sequential(*self.conv_encoder_block)
        
        # conv_output_dim is the output dimension of conv encoder
        conv_output_dim = self.infer_conv_output_dim(self.in_channels, self.input_dim)
        
        self.n_codes = poly_n_codes
        self.attention_num_heads = poly_attention_num_heads
        self.codes_attention_num_heads = codes_attention_num_heads

        # the codes
        codes = torch.empty(self.n_codes, embed_dim)
        codes = torch.nn.init.uniform_(codes)
        self.codes = torch.nn.Parameter(codes)

        # attention for the codes
        self.code_attention = PolyBasicAttention(poly_type='codes', n_codes=self.n_codes, dim=2, attn='basic', get_weights=False)

        # The final attention (the one that takes the candidate as key)
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, 
                                               num_heads=self.attention_num_heads, 
                                               dropout=0.0)
#         self.attention = PolyBasicAttention(poly_type='codes', n_codes=self.n_codes, dim=2, attn='basic', get_weights=False)

        
        self.att_encoder_block = nn.ModuleList()
        
        # conv encoder를 거쳐나온 데이터의 dim과 embed_dim이 다를 경우 맞춰준다
        if conv_output_dim != embed_dim:
            self.att_encoder_block.append(nn.Linear(conv_output_dim, embed_dim))
        
        # SelfAttentionEncoder * num_att_layers
        for i in range(num_att_layers):
            self.att_encoder_block.append(
                torch.nn.TransformerEncoderLayer(d_model=embed_dim, 
                                                 nhead=self.attention_num_heads, 
                                                 dim_feedforward=embed_dim*4, 
                                                 dropout=dropout, 
                                                 activation='gelu')
            )
                            
        self.att_encoder_block = nn.Sequential(*self.att_encoder_block)

            
    def attend(self, attention_layer, queries, keys, values, mask):
        """
        Apply attention.
        :param attention_layer:
            nn.Module attention layer to use for the attention
        :param queries:
            the queries for attention
        :param keys:
            the keys for attention
        :param values:
            the values for attention
        :param mask:
            mask for the attention keys
        :return:
            the result of applying attention to the values, with weights computed
            wrt to the queries and keys.
        """
        if keys is None:
            keys = values
        if isinstance(attention_layer, PolyBasicAttention):
            return attention_layer(queries, keys, values=values, mask_ys=mask)
        elif isinstance(attention_layer, nn.MultiheadAttention):
            return attention_layer(query=queries, key=keys, value=values, attn_mask=mask)[0]

        else:
            raise Exception('Unrecognized type of attention')

    def encode(self, xs):
        # x_raw = [current song, next song]
        # next: candidate
        cs, ns = xs
        
        # padded tensor
        # B, C, T, F
        bsz, in_channels, max_seq_len, _ = ns.size()

        # cand mfcc를 conv encoder를 거친 emb
        cand_emb = self.conv_encoder_block(ns)

        bsz, _, output_seq_len, _ = cand_emb.size()

        # (B, C, T, F) -> (B, T, C, F) -> (B, T, C*F)
        cand_emb = cand_emb.transpose(1, 2)
        cand_emb = cand_emb.contiguous().view(bsz, output_seq_len, -1)

        # transformer encoder
        cand_emb = self.att_encoder_block(cand_emb)

        # reduction : first, avg, max
        if self.reduction_type=='first':
            cand_emb = cand_emb[:,0,:]
        elif self.reduction_type == 'avg':
            cand_emb = torch.mean(cand_emb, dim=1)
        elif self.reduction_type == 'max':
            cand_emb = torch.max(cand_emb, dim=1).values
        else:
            raise KeyError('Not Registered reduction_type. Capable options : first, avg, and max')
        cand_emb = cand_emb.view(cand_emb.size()[0], 1, cand_emb.size()[1])

        # ctxt mfcc를 conv encoder를 거친 emb
        ctxt_out = self.conv_encoder_block(cs)
        b, c, t, f = ctxt_out.size()

        # (B, C, T, F) -> (B, T, C, F) -> (B, T, C*F)
        ctxt_out = ctxt_out.transpose(1, 2)
        ctxt_out = ctxt_out.contiguous().view(b, t, -1)

        # transformer encoder
        ctxt_out = self.att_encoder_block(ctxt_out)

        return ctxt_out, cand_emb
                
    def forward(self, x_raw=None, x_rep=None):
        
        if x_raw is not None:
            return self.encode(x_raw)
        elif x_rep is not None:
            ctxt_out, cand_emb = x_rep

            # m개 만큼 context code를 반복
            # ctxt_out 값과 code를 내적한 값들의 softmax한 벡터 (w_1,...,w_m)를 이전 레이어 결과값(ctxt_out)과 곱해서 합한다.
            # 이 값이 m개의 global context features
            bsz = cand_emb.size(0)
    #             print(f'cand_emb after encoding : {cand_emb.size()}')
    #             print(f'cand_emb : {cand_emb}\n')
    #             print(f'ctxt_out after encoding : {ctxt_out.size()}')
            global_ctxts = self.attend(attention_layer=self.code_attention , 
                                       queries=self.codes.repeat(bsz, 1, 1), 
                                       keys=ctxt_out,
                                       values=ctxt_out, 
                                       mask=None) 
    #             print(f'global_ctxts : {global_ctxts.size()}')

            # torch.nn.MultiheadAttention은 (seq_len, bsz, embed_dim) 으로 입력값을 받아서 수정
            global_ctxts = global_ctxts.transpose(0,1)

    #             print(f'global_ctxts : {global_ctxts.size()}')
    #             print(f'cand_emb : {cand_emb}\n')
            cand_emb = cand_emb.transpose(0,1)

            # m개의 global context features를 cand_emb와 내적한 값을 softmax한 벡터를 (w_1,...,w_m)라 할 때, 이 가중치 값과 global contxt features를 곱해서 합한다.
            # 이 값이 최종 ctxt_emb
            ctxt_emb = self.attend(attention_layer=self.attention ,
                                   queries=cand_emb, 
                                   keys=global_ctxts,
                                   values=global_ctxts,
                                   mask=None)
    #             ctxt_emb = ctxt_emb.transpose(0, 1)
    #             print(f'ctxt_emb size : {ctxt_emb.size()}')

    #             print(f'ctxt_emb : {ctxt_emb}\n')

            # score: cand_emb와 ctxt_emb 간 cosine similarity값 (반환값)
            scores = torch.sum(ctxt_emb * cand_emb, -1)
            scores = scores.view(1, -1)
    #             print(f'scores : {scores.size()}')
    #             print(f'scores : {scores}')
            return scores
        else:
            raise Exception('not support Operation')
    
    def infer_conv_output_dim(self, in_channels, input_dim):
        sample_seq_len = 200
        sample_bsz = 10
        x = torch.randn(sample_bsz, in_channels, sample_seq_len, input_dim)
        for i, _ in enumerate(self.conv_encoder_block):
            x = self.conv_encoder_block[i](x)
        # (B, C, T, F) -> (B, T, C, F) -> (B, T, C*F)
        x = x.transpose(1, 2)
        mb, seq = x.size()[:2]
        return x.contiguous().view(mb, seq, -1).size(-1)


In [18]:
model = AudioEncoder(
                     poly_n_codes=64, # m, the number of global contxt features
                     poly_attention_num_heads=8, 
                     codes_attention_num_heads=2,
                     num_att_layers=6,
                     embed_dim=512, 
                     input_feat_per_channel=40, # feature vector dimension
                     num_conv_block=2,
                     num_conv_layers=2, 
                     in_channels=1, 
                     out_channels=32,
                     conv_kernel_size=3, 
                     pooling_kernel_size=2,
                     layer_norm=True,
                     reduction_type='first'
)

--------------------------------------------

In [19]:
from ignite.engine import Events #, create_supervised_trainer, create_supervised_evaluator
# from ignite.metrics import Loss

In [20]:
def train_batch(data, bsz, shuffle=True):
    #TODO encoding 후 np array 혹은 cpu에 할당하고 나서 gpu 사용량을 줄이는 시도
    cs_rep, ns_rep, labels = data
    length = cs_rep.size()[0]

    for i in range(0, length, bsz):
        cs_batch = cs_rep[i:i+bsz]
        ns_batch = ns_rep[i:i+bsz]
        label_batch = labels[i:i+bsz]

        if shuffle:
            indexes = torch.randperm(bsz)              
            yield [[cs_batch[indexes], ns_batch[indexes]], label_batch[indexes]]
        else:
            yield [[cs_batch, ns_batch], label_batch]    

In [21]:
def get_data_loaders(raw_data, train_batch_size, val_batch_size=None, val_ratio=0.2):
    if val_ratio > 0:
        train_len = int((1-val_ratio)*len(labels))
        trainset = [cs[:train_len], ns[:train_len], labels[:train_len]]
        valset = [cs[train_len:], ns[train_len:], labels[train_len:]]
    else:
        trainset = raw_data
        valset = None
    t_total = len(trainset[2])
    
    train_loader = train_batch(trainset, train_batch_size, shuffle=True)
    if valset is not None:
        if val_batch_size is None:
            val_batch_size = train_batch_size
        val_loader = train_batch(valset, val_batch_size, shuffle=False)
        return t_total, train_loader, val_loader
    return t_total, train_loader, None

In [22]:
from typing import Callable, Optional, Sequence, Tuple, Union
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import Metric, reinit__is_reduced, sync_all_reduce


class RetrievalAccuracy(Metric):
    def __init__(
        self,
        output_transform: Callable = lambda x:x,
        device: Union[str, torch.device] = torch.device('cpu'),
    ):
        super(RetrievalAccuracy, self).__init__(output_transform=output_transform, device=device)
#         self._device=device
    
    @reinit__is_reduced
    def reset(self) -> None:
        self._num_correct = torch.tensor(0, device=self._device)
        self._len = 0
        super(RetrievalAccuracy, self).reset()
    
    @reinit__is_reduced
    def update(self, output: Sequence[torch.Tensor]) -> None:
        logit, y = output[0].detach(), output[1].detach()
        
        y_pred = torch.argmax(logit, axis=1).item()
        y = torch.argmax(y).item()
        
        print(f'y_pred {y_pred}, y {y}')
        
        if y_pred == y:
            self._num_correct += torch.tensor(1).to(self._device)
            
        self._len += 1
        print(f'_len : {self._len}')
    
    @sync_all_reduce("_len", "_num_correct")
    def compute(self) -> float:
        if self._len == 0:
            return 0.0
#             raise NotComputableError('Accuracy must have at least one example before it can be computed.')
        return self._num_correct.item() / self._len
    
    
# class Loss(Metric):
#     def __init__(self):
#         super(RetrievalAccuracy, self).__init__()
#         self.reset()
        
#     def reset(self):
#         self._cost = 0
#         self._len = 0
        
#     def update(self, output):
#         logit, y = output
                
#         # label
#         y = torch.LongTensor([torch.argmax(y).item()]).to(device)
            
#         loss = criterion(logit, y)
        
#         if y_pred == y:
#             self._num_correct += 1
            
#         self._len += 1
    
#     def compute(self):
#         if self._len == 0:
#             raise ValueError('length : 0')
#         return self._num_correct / self._len

In [23]:
from typing import Callable, Dict, Sequence, Tuple, Union, cast

import torch

from ignite.exceptions import NotComputableError
from ignite.metrics.metric import Metric, reinit__is_reduced, sync_all_reduce

__all__ = ["Loss"]


class Loss(Metric):
    """
    ref : https://github.com/pytorch/ignite/blob/786aea82725ef114737913f76599665bc15b5de5/ignite/metrics/loss.py
    
    Calculates the average loss according to the passed loss_fn.
    Args:
        loss_fn: a callable taking a prediction tensor, a target
            tensor, optionally other arguments, and returns the average loss
            over all observations in the batch.
        output_transform: a callable that is used to transform the
            :class:`~ignite.engine.engine.Engine`'s ``process_function``'s output into the
            form expected by the metric.
            This can be useful if, for example, you have a multi-output model and
            you want to compute the metric with respect to one of the outputs.
            The output is expected to be a tuple `(prediction, target)` or
            (prediction, target, kwargs) where kwargs is a dictionary of extra
            keywords arguments. If extra keywords arguments are provided they are passed to `loss_fn`.
        batch_size: a callable taking a target tensor that returns the
            first dimension size (usually the batch size).
        device: specifies which device updates are accumulated on. Setting the
            metric's device to be the same as your ``update`` arguments ensures the ``update`` method is
            non-blocking. By default, CPU.
    Attributes:
        required_output_keys: dictionary defines required keys to be found in ``engine.state.output`` if the
            latter is a dictionary. Default, ``("y_pred", "y", "criterion_kwargs")``. This is useful when the
            criterion function requires additional arguments, which can be passed using ``criterion_kwargs``.
            See notes below for an example.
    Note:
        Let's implement a Loss metric that requires ``x``, ``y_pred``, ``y`` and ``criterion_kwargs`` as input
        for ``criterion`` function. In the example below we show how to setup standard metric like Accuracy
        and the Loss metric using an ``evaluator`` created with
        :meth:`~ignite.engine.create_supervised_evaluator` method.
        .. code-block:: python
            import torch
            import torch.nn as nn
            from torch.nn.functional import nll_loss
            from ignite.metrics import Accuracy, Loss
            from ignite.engine import create_supervised_evaluator
            model = ...
            criterion = nll_loss
            metrics = {
                "Accuracy": Accuracy(),
                "Loss": Loss(criterion)
            }
            # global criterion kwargs
            criterion_kwargs = {...}
            evaluator = create_supervised_evaluator(
                model,
                metrics=metrics,
                output_transform=lambda x, y, y_pred: {
                    "x": x, "y": y, "y_pred": y_pred, "criterion_kwargs": criterion_kwargs}
            )
            res = evaluator.run(data)
    """

    required_output_keys = ("y_pred", "y", "criterion_kwargs")

    def __init__(
        self,
        loss_fn: Callable,
        output_transform: Callable = lambda x: x,
        batch_size: Callable = len,
        device: Union[str, torch.device] = torch.device("cpu"),
    ):
        super(Loss, self).__init__(output_transform, device=device)
        self._loss_fn = loss_fn
        self._batch_size = batch_size
        self._device = device

    @reinit__is_reduced
    def reset(self) -> None:
        self._sum = torch.tensor(0.0, device=self._device)
        self._num_examples = 0

    @reinit__is_reduced
    def update(self, output: Sequence[Union[torch.Tensor, Dict]]) -> None:
        print(f'loss device : {self._device}')
        if len(output) == 2:
            y_pred, y = cast(Tuple[torch.Tensor, torch.Tensor], output)
            kwargs = {}  # type: Dict
        else:
            y_pred, y, kwargs = cast(Tuple[torch.Tensor, torch.Tensor, Dict], output)
        
        # label
        y = torch.LongTensor([torch.argmax(y).item()]).to(self._device)

        average_loss = self._loss_fn(y_pred, y, **kwargs).detach()

        if len(average_loss.shape) != 0:
            raise ValueError("loss_fn did not return the average loss.")

#         n = self._batch_size(y)
        self._sum += average_loss.to(self._device)
        self._num_examples += 1

    @sync_all_reduce("_sum", "_num_examples")
    def compute(self) -> float:
        if self._num_examples == 0:
            return 0
#             raise NotComputableError("Loss must have at least one example before it can be computed.")
        return self._sum.item() / self._num_examples

In [24]:
def val_step(evaluator, batch):
    # ...
    s = evaluator.state
    print(
        f"{s.epoch}/{s.max_epochs} : {s.iteration} - {batch:.3f}"
    )
    

In [25]:
import os
from collections.abc import Mapping
from typing import Any, Callable, Dict, Optional, Sequence, Tuple, Union
from ignite.handlers.checkpoint import ModelCheckpoint
from ignite.utils import setup_logger
from tqdm import tqdm
from ignite.engine import Events, Engine
from ignite.utils import convert_tensor

In [26]:
def _prepare_batch(
    batch: Sequence[torch.Tensor], device: Optional[Union[str, torch.device]] = None, non_blocking: bool = False
) -> Tuple[Union[torch.Tensor, Sequence, Mapping, str, bytes], ...]:
    """Prepare batch for training: pass to a device with options.
    """
    x, y = batch
    return (
        convert_tensor(x, device=device, non_blocking=non_blocking),
        convert_tensor(y, device=device, non_blocking=non_blocking),
    )

def evaluation_step(
    model: torch.nn.Module,
    device: Optional[Union[str, torch.device]] = None,
    non_blocking: bool = False,
    prepare_batch: Callable = _prepare_batch,
    output_transform: Callable = lambda x, y, y_pred: (y_pred, y),
) -> Callable:
    def evaluate_step(engine: Engine, batch: Sequence[torch.Tensor]) -> Union[Any, Tuple[torch.Tensor]]:
        model.eval()
        with torch.no_grad():
            x, y = prepare_batch(batch, device=device, non_blocking=non_blocking)
            
            x_rep = model(x_raw=x)
            y_pred = model(x_rep=x_rep)    
            
            return output_transform(x, y, y_pred)

    return evaluate_step

def create_evaluator(
    model: torch.nn.Module,
    metrics: Optional[Dict[str, Metric]] = None,
    device: Optional[Union[str, torch.device]] = None,
    non_blocking: bool = False,
    prepare_batch: Callable = _prepare_batch,
    output_transform: Callable = lambda x, y, y_pred: (y_pred, y),
    amp_mode: Optional[str] = None,
) -> Engine:
    
    '''
    ref : https://github.com/pytorch/ignite/blob/master/ignite/engine/__init__.py > create_supervised_evaluator()
    '''
    metrics = metrics or {}
    
    eval_step = evaluation_step(model, device, non_blocking, prepare_batch, output_transform)
    
    evaluator = Engine(eval_step)
    
    for name, metric in metrics.items():
        metric.attach(evaluator, name)
        
    return evaluator

In [27]:

def create_trainer(model, optimizer, criterion, device, config=None, lr_scheduler=None):

    # Define any training logic for iteration update
    def train_step(engine, batch):
        cs, ns, y = batch[0][0].to(device),batch[0][1].to(device), batch[1].to(device)

        model.train()
        
        x_rep = model(x_raw=[cs, ns])
        y_pred = model(x_rep=x_rep)
        
        # label
        y = torch.LongTensor([torch.argmax(y).item()]).to(device)
            
        loss = criterion(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if lr_scheduler is not None:
            lr_scheduler.step()

        return loss.item()

    # Define trainer engine
    trainer = Engine(train_step)

    return trainer


    

def train(data, model, train_batch_size, val_batch_size, output_dir, model_filename, device, epochs, lr, betas=(0.9, 0.999), weight_decay: float = 0.01):
    # epochs, batch_size, output_dir, lr, betas=(0.9, 0.999), weight_decay: float = 0.01, val_ratio=0.2, fp16=False, fp16_opt_level='O1'
    
    t_total, train_loader, val_loader = get_data_loaders(data, train_batch_size, val_batch_size)
    
    log_interval = int((t_total / train_batch_size) // 3)
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)

#     trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
    trainer = create_trainer(model, optimizer, criterion, device)
    trainer.logger = setup_logger("trainer")
    evaluator = create_evaluator(model,
                                 metrics={
                                     'accuracy': RetrievalAccuracy(device=device),
                                     'loss': Loss(criterion, device=device),
                                },
                                 device=device)
    evaluator.logger = setup_logger("evaluator")
    
    pbar = tqdm(initial=0, leave=False, total=t_total, desc=f"ITERATION - loss: {0:.2f}")
    
    model.to(device)
    
    # execution after every log_interval
    @trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
    def log_training_loss(engine):        
        pbar.desc = f"Epoch[{engine.state.epoch}] ITER {engine.state.iteration} - Loss: {engine.state.output:.2f}"
        pbar.update(log_interval)
 
    # execution after every training epoch
    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(engine):
        # evaluate on training set
        pbar.refresh()
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        avg_accuracy = metrics["accuracy"]
        avg_loss = metrics["loss"]
        tqdm.write(f"Training Results - Epoch: {engine.state.epoch} Avg accuracy: {avg_accuracy:.2f} Avg loss: {avg_loss:.2f}")
        
#         fp = 
#         torch.save(model.state_dict(), fp)


    # execution after every epoch
    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(engine):
        # evaluate test(validation) set
        evaluator.run(val_loader)
        metrics = evaluator.state.metrics
        avg_accuracy = metrics["accuracy"]
        avg_loss = metrics["loss"]
        tqdm.write(f"Validation Results - Epoch: {engine.state.epoch} Avg accuracy: {avg_accuracy:.2f} Avg loss: {avg_loss:.2f}")
        pbar.n = pbar.last_print_n = 0
        
    @trainer.on(Events.EPOCH_COMPLETED | Events.COMPLETED)
    def log_time(engine):
        tqdm.write(f"{trainer.last_event_name.name} took { trainer.state.times[trainer.last_event_name.name]} seconds")
    
    ckpt = ModelCheckpoint(output_dir, model_filename, n_saved=3, create_dir=True)
    trainer.add_event_handler(Events.EPOCH_COMPLETED(every=log_interval), ckpt, {'mymodel': model})
    
    trainer.run(train_loader, max_epochs=epochs)
    pbar.close()
    
    

In [28]:
# Hyperparameters
epochs = 100 # epoch
batch_size=32 # batch size for training
lr=1e-5 # learning rate
device='cuda:1'
train([cs, ns, labels], 
      model, 
      train_batch_size=batch_size, 
      val_batch_size=batch_size, 
      output_dir='.', 
      model_filename='ignite-poly-64-0904', 
      device=device, 
      epochs=epochs, 
      lr=lr)

Epoch[1] ITER 8 - Loss: 12.46:   3%|█                                  | 8/256 [00:07<03:55,  1.05it/s]2021-09-04 16:15:02,679 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:02,679 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:02,680 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   3%|█                                  | 8/256 [00:07<03:55,  1.05it/s]2021-09-04 16:15:02,682 evaluator INFO: Engine run starting with max_epochs=1.


Training Results - Epoch: 1 Avg accuracy: 0.00 Avg loss: 0.00
y_pred 10, y 0
_len : 1
loss device : cuda:1


2021-09-04 16:15:03,133 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,134 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,136 trainer INFO: Epoch[1] Complete. Time taken: 00:00:08
/home/super/.pyenv/versions/3.8.0/envs/waterffle/lib/python3.8/site-packages/ignite/engine/engine.py:793: UserWarning: Data iterator can not provide data anymore but required total number of iterations to run is not reached. Current iteration: 8 vs Total iterations to run : 800
  warnings.warn(
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,137 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,138 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,138 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 -

2021-09-04 16:15:03,254 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,258 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,258 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,258 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,260 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,261 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,261 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,263 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,264 evaluator INFO: Epoch[1]

Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,311 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,312 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,312 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,313 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,314 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,314 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,317 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,318 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,318 evaluator INFO: Engine run

...

Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,705 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,705 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,705 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,708 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,709 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,709 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,710 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,711 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,711 evaluator INFO: Engine run

Training Results - Epoch: 57 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 57 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.8835067749023438e-05 seconds
Training Results - Epoch: 58 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 58 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.9550323486328125e-05 seconds
Training Results - Epoch: 59 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 59 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.9073486328125e-05 seconds
Training Results - Epoch: 60 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 60 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.9311904907226562e-05 seconds
Training Results - Epoch: 61 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 61 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.9311904907226562e-05 seconds
Training Results - Epoch: 62 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 62 Avg accur

Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,736 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,737 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,737 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,738 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,739 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,739 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,742 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,743 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,743 evaluator INFO: Engine run

Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,794 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,794 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,795 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,797 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,798 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,799 evaluator INFO: Engine run complete. Time taken: 00:00:00
Epoch[1] ITER 8 - Loss: 12.46:   0%|                                   | 0/256 [00:08<04:03,  1.05it/s]2021-09-04 16:15:03,800 evaluator INFO: Engine run starting with max_epochs=1.
2021-09-04 16:15:03,800 evaluator INFO: Epoch[1] Complete. Time taken: 00:00:00
2021-09-04 16:15:03,801 evaluator INFO: Engine run

Training Results - Epoch: 90 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 90 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.9311904907226562e-05 seconds
Training Results - Epoch: 91 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 91 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.8835067749023438e-05 seconds
Training Results - Epoch: 92 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 92 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.9311904907226562e-05 seconds
Training Results - Epoch: 93 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 93 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.9073486328125e-05 seconds
Training Results - Epoch: 94 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 94 Avg accuracy: 0.00 Avg loss: 0.00
EPOCH_COMPLETED took 1.9550323486328125e-05 seconds
Training Results - Epoch: 95 Avg accuracy: 0.00 Avg loss: 0.00
Validation Results - Epoch: 95 Avg accur

In [ ]:
tdata = [cs.to(device_num), ns.to(device_num), labels.to(device_num)]
# batch_size, C, H, W = 32, 1, 28, 28
# tdata = [torch.randn(batch_size, C, H, W).to(1),torch.randn(batch_size, C, H, W).to(1), torch.randn(batch_size).to(1)]

# Hyperparameters
epochs = 100 # epoch
batch_size=32 # batch size for training
lr=1e-5 # learning rate

# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#             optimizer, mode='min', factor=0.2, threshold=0.01, patience=5)

train(model=model.to(device_num), 
      raw_data=tdata, 
      epochs=epochs, 
      batch_size=batch_size, 
      output_dir='.', 
      lr=lr,
      val_ratio=0.2, 
      fp16=False, 
      fp16_opt_level='O1')

In [12]:
def eval_model(dataloader, criterion):
    # layer 중 dropout layer와 같이 학습 시에는 사용하는데 inference할 때는 사용하지 않는 경우를 구분해주기 위함
    model.eval()
    
    n_steps = 0
    eval_loss = 0    
    acc = 0

    # autograd engine을 끔(gradient 계산하지 않음) --> 메모리 사용량을 줄이고 연산 속도를 높이기 위함
    with torch.no_grad():
        # input은 이미 1차 임베딩은 되어 있는 것을 가져왔다고 가정한다.
        for idx, (batch_cs, batch_ns, batch_label) in enumerate(dataloader):
            n_steps+=1
            # mfcc embedding encoder
            cs_rep, ns_rep, label = model(x_raw=[batch_cs, batch_ns, batch_label])

            batch_label = torch.LongTensor([torch.argmax(label).item()]).to(device_num)
            scores = model(x_rep=[cs_rep, ns_rep])
            
#             print(f'eval score : {scores}')
                
            loss = criterion(scores, batch_label)

            eval_loss += loss.item()
            
#             print(f'predict : {torch.argmax(scores, axis=1)}')
#             print(f'label.  : {batch_label}')
            if torch.argmax(scores, axis=1) == batch_label:
                acc += 1

    results = {
        'eval_accuracy': acc / n_steps,
        'eval_loss': eval_loss / n_steps
    }
    return results
    

In [25]:
device_num = 1

In [23]:
import time
import os
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

def train(model, raw_data, epochs, batch_size, output_dir, lr, betas=(0.9, 0.999), weight_decay: float = 0.01, val_ratio=0.2, fp16=False, fp16_opt_level='O1'):
#     no_decay = ["bias", "LayerNorm.weight"]
    
#     optimizer_grouped_parameters = [
#         {
#             "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#             "weight_decay": weight_decay,
#         },
#         {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
#     ]
    

    
    # fp16=True면 amp를 통한 mixed preicision training을 한다는 의미
    # 사용 조건 : Volta 이상의 nvidia 그래픽 카드(v100, rtx2080ti, 등)
    if fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=fp16_opt_level)
    
    def train_batch(data, bsz, shuffle=True):
        cs_rep, ns_rep, labels = data
        length = cs_rep.size()[0]
        
        for i in range(0, length, bsz):
            cs_batch = cs_rep[i:i+bsz]
            ns_batch = ns_rep[i:i+bsz]
            label_batch = labels[i:i+bsz]
            
            if shuffle:
                indexes = torch.randperm(bsz)              
                yield [cs_batch[indexes], ns_batch[indexes], label_batch[indexes]]
            else:
                yield [cs_batch, ns_batch, label_batch]    
    state_save_path = os.path.join(output_dir, '{}_{}_{}_pytorch_model.bin'.format('polyencoder', '64', '0815'))
        
    best_eval_loss = float('inf')
    best_test_loss = float('inf')
    
    cs, ns, labels = raw_data
        
    init_time = time.time()
    # 재사용 가능한 임베딩들은 미리 뽑기
    trainset = None
    valset = None

    if val_ratio > 0:
        train_len = int((1-val_ratio)*len(labels))
        trainset = [cs[:train_len], ns[:train_len], labels[:train_len]]
        valset = [cs[train_len:], ns[train_len:], labels[train_len:]]
    else:
        trainset = raw_data

    eval_freq = (len(trainset[2]) / batch_size) // 3
    
    t_total = len(trainset[0])
    
#     optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
#     scheduler = get_linear_schedule_with_warmup(
#         optimizer, num_warmup_steps=100, num_training_steps=t_total
#     )
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.2, threshold=0.01, patience=5)

    # Set the learning rate of each parameter group using a cosine annealing schedule,
    # https://pytorch.org/docs/1.8.0/optim.html?highlight=cosineannealingwarmrestarts#torch.optim.lr_scheduler.CosineAnnealingWarmRestarts
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    # stepRL
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=eval_freq, gamma=0.95)

        
    batch_labels = []
    scores = []
    global_step = 0
    #TODO : 나중에 tqdm으로 바꾸기
    for epoch in range(1, epochs+1):
        start_time = time.time()
        train_batches = train_batch(trainset, batch_size)
        
        total_loss = 0.0
        accuracy = 0.0
        local_step = 0
        for idx, (batch_cs, batch_ns, batch_label) in enumerate(train_batches):
            model.train()
            
            # mfcc embedding encoder
            cs_rep, ns_rep, label = model(x_raw=[batch_cs, batch_ns, batch_label])
            
            # label
            batch_label = torch.LongTensor([torch.argmax(batch_label).item()]).to(device_num)
            
            # dot product
            scores = model(x_rep=[cs_rep, ns_rep])
#             print(f'scores : {scores}')

            # cross entropy
            loss = criterion(scores, batch_label)

            total_loss += loss.item()
            
            predict = torch.argmax(scores, axis=1)
#             print('predict : ',predict.item())
#             print(f'label : {batch_label.item()}\n')
#             if predict == torch.argmax(batch_label).item():
            if predict == batch_label:
                accuracy += 1
#                 print(f'-- correct : {accuracy}\n')
            
            if torch.isnan(loss):
                print(f'scores : {scores}')
                exit()
            
            if fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            optimizer.step()
#             scheduler.step()

            # clear gradient
            optimizer.zero_grad()

            
#             model.zero_grad()
            
#             if global_step and global_step % eval_freq == 0:
#                 if valset is not None:
#                     val_batches = train_batch(valset, batch_size, shuffle=False)

#                 eval_results = eval_model(val_batches, criterion)
#                 scheduler.step(eval_results['eval_loss'])

#                 print(f'Global Step {global_step} Eval loss : {eval_results["eval_loss"]}, Eval accuracy : {eval_results["eval_accuracy"]}')
#                 if eval_results["eval_loss"] < best_eval_loss:
#                     best_eval_loss = eval_results['eval_loss']
#                     # save model.state_dict()
#                     torch.save(model.state_dict(), state_save_path)
#                     print(f'[Saving at] {state_save_path}')

            
            global_step += 1
            local_step += 1

            #print(f'batch {idx} | Loss {loss.item()}')
        
        total_loss = total_loss / local_step
        train_accuracy = accuracy / local_step
        print(f'\n[Epoch {epoch}] | train_cost {total_loss} | train_accuracy {train_accuracy} | time {time.time()-start_time}')
        
        if valset is not None:
            val_batches = train_batch(valset, batch_size, shuffle=False)
        eval_results = eval_model(val_batches, criterion)
        print(f'Eval loss : {eval_results["eval_loss"]}, Eval accuracy : {eval_results["eval_accuracy"]}')
        if eval_results["eval_loss"] < best_eval_loss:
            best_eval_loss = eval_results['eval_loss']
            # save model.state_dict()
            torch.save(model.state_dict(), state_save_path)
#             print(f'[Saving at] {state_save_path}')
        print('---------------------------------\n')
            

In [ ]:
# RuntimeError: Expected 4-dimensional input for 4-dimensional weight [256, 1, 3, 3], but got 3-dimensional input of size [320, 40, 2800] instead
# tdata = [now_mfcc_list.to(1), next_mfcc_list.to(1), label_list.to(1)]
tdata = [cs.to(device_num), ns.to(device_num), labels.to(device_num)]
# batch_size, C, H, W = 32, 1, 28, 28
# tdata = [torch.randn(batch_size, C, H, W).to(1),torch.randn(batch_size, C, H, W).to(1), torch.randn(batch_size).to(1)]

# Hyperparameters
epochs = 100 # epoch
batch_size=32 # batch size for training
lr=1e-5 # learning rate

# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#             optimizer, mode='min', factor=0.2, threshold=0.01, patience=5)

train(model=model.to(device_num), 
      raw_data=tdata, 
      epochs=epochs, 
      batch_size=batch_size, 
      output_dir='.', 
      lr=lr,
      val_ratio=0.2, 
      fp16=False, 
      fp16_opt_level='O1')


[Epoch 1] | train_cost 2.9114290095865725 | train_accuracy 0.1625 | time 74.13653993606567
Eval loss : 6.218299305438995, Eval accuracy : 0.0
---------------------------------


[Epoch 2] | train_cost 2.331125350110233 | train_accuracy 0.2 | time 74.33295392990112
Eval loss : 6.199160850048065, Eval accuracy : 0.0
---------------------------------


[Epoch 3] | train_cost 2.400858477782458 | train_accuracy 0.2625 | time 74.48530435562134
Eval loss : 6.528730642795563, Eval accuracy : 0.0
---------------------------------


[Epoch 4] | train_cost 2.261480643879622 | train_accuracy 0.2625 | time 74.50722742080688
Eval loss : 6.581989181041718, Eval accuracy : 0.0
---------------------------------


[Epoch 5] | train_cost 2.355602344824001 | train_accuracy 0.275 | time 74.5271053314209
Eval loss : 6.960526299476624, Eval accuracy : 0.0
---------------------------------


[Epoch 6] | train_cost 2.4346638480201364 | train_accuracy 0.175 | time 74.52601099014282
Eval loss : 6.63511180877685

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F


#             2. max 값으로 대표값 정해서 cross entropy 32,32 -> 32, 1 -> 1, 32
#             3. mean 값으로 대표값 정해서 cross entropy 32, 32 -> 32,1 -> 1, 32

x = torch.randn((32,32))
print(f'original : {x.size()}\n{x}')
# x = torch.max(x, dim=-1)
# x = x.values.view(1, -1)
# x.size()

x = torch.mean(x, dim=-1)
x = x.view(1, -1)
x.size()

# x = F.softmax(x)
# print(x.size())
# y = torch.LongTensor([1])
# y.size()

original : torch.Size([32, 32])
tensor([[ 2.4609,  2.9383, -0.0992,  ...,  1.5904, -0.4699, -0.3806],
        [ 1.4856, -1.7485, -0.7887,  ...,  0.5148, -1.7101,  1.5381],
        [-1.4856,  0.5917, -0.4003,  ..., -0.6114, -0.2885,  1.0095],
        ...,
        [ 0.1101,  0.3752, -1.4103,  ..., -0.7080, -2.1661,  1.6945],
        [ 0.6835,  0.1333, -1.1536,  ...,  0.2719,  0.2294, -0.0227],
        [-1.2907,  1.2089, -0.3617,  ...,  0.3123, -1.5061, -0.4800]])


torch.Size([1, 32])

In [ ]:
cross_entropy_loss = torch.nn.CrossEntropyLoss()
print(cross_entropy_loss(x, y))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
x = torch.Tensor([[2, 0.805, 0.6393]])
x = F.softmax(x)
print(x.size())
y = torch.LongTensor([0])
y.size()

In [ ]:
cross_entropy_loss = torch.nn.CrossEntropyLoss()
print(cross_entropy_loss(x, y))

In [ ]:
y1 = torch.LongTensor([1])
print(cross_entropy_loss(x, y1))

In [2]:
y2 = torch.LongTensor([2])
print(cross_entropy_loss(x, y2))

NameError: name 'torch' is not defined

In [ ]:
a = torch.randn(4)
print(a)
a = torch.LongTensor([torch.argmax(a).item()])
print(a)
a.size()

In [ ]:
y = torch.LongTensor([1])
y.size()

In [ ]:
x = torch.FloatTensor([[2.0959e-02, 2.0501e-04, 1.5565e-02, 5.2595e-02, 1.3077e-05, 1.2790e-03,
         3.7969e-01, 3.6344e-05, 1.1723e-01, 5.2833e-02, 2.7906e-05, 2.4717e-04,
         4.2139e-05, 3.3961e-04, 1.4989e-05, 1.3840e-03, 1.6910e-02, 1.2072e-03,
         2.4667e-02, 2.1032e-03, 2.3573e-01, 3.0013e-02, 6.1480e-04, 1.3032e-02,
         7.4176e-03, 1.4105e-02, 5.0791e-04, 9.9195e-04, 2.5835e-05, 7.1133e-03,
         6.1536e-04, 2.4788e-03]])

In [3]:
torch.argmax(x)

NameError: name 'torch' is not defined

In [ ]:
y = torch.LongTensor([6])

In [ ]:
x.size()

In [ ]:
for i in range(x.size()[1]):
    y = torch.LongTensor([i])
    cross_entropy_loss = torch.nn.CrossEntropyLoss()
    print(cross_entropy_loss(x, y))